# Fine tuning Hyperparameters

In [1]:
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import Adam
import keras
from keras.layers import Dense,Dropout
from keras.callbacks import EarlyStopping #remember this

In [2]:
#loading and handling the data
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train=x_train.reshape(x_train.shape[0],28*28).astype('float32')/255
x_test=x_test.reshape(x_test.shape[0],28*28).astype('float32')/255
y_train=keras.utils.to_categorical(y_train,10)
y_test=keras.utils.to_categorical(y_test,10)

In [3]:
def build_model(learning_rate=0.1,dropout_rate=0.2,num_neurons=64):
    model=Sequential()
    model.add(Dense(num_neurons,input_shape=(28*28,),activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_neurons,activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(10,activation='softmax'))#sigmoid for binary softmax for categorical
    optimizer=Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [4]:
learning_rate=[0.01,0.001,0.0001]
dropout_rate=[0.2,0.3,0,4]
num_neurons=[32,64,128]
batch_size=[32,64,128]

In [5]:
#creating validation data
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=42)

In [6]:
best_accuracy=0
best_params=[]
for lr in learning_rate:
    for dr in dropout_rate:
        for nn in num_neurons:
            for bs in batch_size:
                print('-'*50)
                print(f"lr={lr} | dr={dr} | nn={nn} | bs={bs}")
                print('-'*50)
                model=build_model(learning_rate=lr,dropout_rate=dr,num_neurons=nn)
                history=model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=20,batch_size=bs,callbacks=[EarlyStopping(monitor='val_loss',patience=5)])
                accuracy=max(history.history['val_accuracy'])
                print('-'*50)
                print(f"Accuracy : {accuracy}")
                print('-'*50)
                if accuracy>best_accuracy:
                    best_accuracy=accuracy
                    best_params={'lr':lr,'dr':dr,'nn':nn,'bs':bs}

print('-'*50)
print('Best Accuracy : ',accuracy)
print('Best Params : ',best_params)
print('-'*50)

--------------------------------------------------
lr=0.01 | dr=0.2 | nn=32 | bs=32
--------------------------------------------------
Epoch 1/20
1500/1500 [==============================] - 7s 4ms/step - loss: 0.5872 - accuracy: 0.8155 - val_loss: 0.2802 - val_accuracy: 0.9197
Epoch 2/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4671 - accuracy: 0.8609 - val_loss: 0.2726 - val_accuracy: 0.9273
Epoch 3/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4511 - accuracy: 0.8663 - val_loss: 0.2357 - val_accuracy: 0.9309
Epoch 4/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4252 - accuracy: 0.8743 - val_loss: 0.2510 - val_accuracy: 0.9295
Epoch 5/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4175 - accuracy: 0.8790 - val_loss: 0.2348 - val_accuracy: 0.9335
Epoch 6/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4144 - accuracy: 0.8815 - val_loss: 0.2471 - val_accuracy: 0.9332
E

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.